In [2]:
import numpy as np, pandas as pd, plotly.express as px, plotly.graph_objects as go
from scipy.signal import savgol_filter as sgf
from pathlib import Path
import scipy.optimize as opt

from SJ_Haar_CNV import base_search
from SJ_Haar_CNV import decomposition
from tests import test_data

In [3]:
X = np.concatenate([np.repeat (5,180), np.repeat (6,21), np.repeat (5,190), np.repeat (6,190), np.repeat(5,100)])+ np.random.normal(0,1,681)
l = 681
#

In [4]:
basis = base_search.generate_haar_basis(X, p0=.8, length = 21, debug = False)
basis

[[0, 0, 0, (5.360256192676781, 681), (0, 0)],
 [0, 681, 0, (0.03363223063637841, 384), (-0.04359733601012016, 297)],
 [1, 384, 0, (0.04825666562229138, 202), (-0.05382474242486347, 182)],
 [1, 296, 385, (0.041653780732643746, 195), (-0.08083307944156609, 101)],
 [2, 202, 0, (0.045463156716430905, 142), (-0.10835385684082699, 60)],
 [2, 181, 203, (0.12043875831929896, 49), (-0.045620741787613245, 132)],
 [2, 195, 385, (0.13888197538238176, 40), (-0.03673652252050099, 155)],
 [2, 100, 581, (0.16473022472523716, 26), (-0.06010427118353248, 74)],
 [3, 142, 0, (0.04670444233259308, 108), (-0.14972894747801896, 34)],
 [3, 59, 143, (0.08788388496062494, 40), (-0.1896441728097696, 19)],
 [3, 49, 203, (0.2516611478423583, 11), (-0.07947194142390261, 38)],
 [3, 131, 253, (0.16774542658006547, 27), (-0.045162230233094555, 104)],
 [3, 40, 385, (0.2579124793542412, 10), (-0.09456790909655509, 30)],
 [3, 154, 426, (0.042577575162185424, 120), (-0.1515260763124834, 34)],
 [3, 26, 581, (0.325300024316

In [5]:
res = decomposition.decompose (X, basis)

In [6]:
basis1, coefficients, unneccessary_remove_later, rec_signal = res

In [7]:
coefficients[0] = 1

In [8]:
decomposition.generate_function_from_wavelets (coefficients, basis, l)

array([4.10971158, 4.10971158, 4.10971158, 4.10971158, 4.10971158,
       4.10971158, 4.10971158, 5.24057361, 5.24057361, 5.24057361,
       5.24057361, 5.24057361, 5.24057361, 5.24057361, 5.24057361,
       5.24057361, 5.24057361, 5.24057361, 5.24057361, 5.24057361,
       5.24057361, 5.24057361, 5.24057361, 5.24057361, 5.09504724,
       4.95451974, 4.95451974, 4.95451974, 4.95451974, 4.95451974,
       4.95451974, 4.95451974, 4.95451974, 4.95451974, 4.95451974,
       4.95451974, 4.95451974, 4.95451974, 4.95451974, 4.95451974,
       4.95451974, 4.95451974, 4.95451974, 4.69816043, 4.69816043,
       4.69816043, 4.69816043, 4.69816043, 4.69816043, 4.69816043,
       4.69816043, 4.69816043, 5.20651158, 4.9021054 , 4.9021054 ,
       4.9021054 , 4.9021054 , 4.9021054 , 4.9021054 , 4.9021054 ,
       4.9021054 , 4.9021054 , 4.9021054 , 4.9021054 , 4.9021054 ,
       4.9021054 , 5.30798031, 5.03466519, 5.03466519, 5.03466519,
       5.03466519, 5.03466519, 5.03466519, 5.03466519, 5.03466

In [9]:
fig = go.Figure()
#fig.add_trace(go.Scatter(x=df['location'], y=df['raw'], name='Raw Signal', mode='markers',marker=markers))
fig.add_trace(go.Scatter(y = X, mode = 'markers'))
fig.add_trace (go.Scatter (y = decomposition.generate_function_from_wavelets (coefficients, basis, l)))
ind = np.array([0,   1,3])
#allind = np.arange(len(basis))
fig.add_trace (go.Scatter (y =decomposition.generate_function_from_wavelets (coefficients[ind], [basis[i] for i in ind], l)))

In [10]:
px.plot (y = X - decomposition.generate_function_from_wavelets (coefficients[ind], [basis[i] for i in ind], l))

AttributeError: module 'plotly.express' has no attribute 'plot'

In [15]:
fig = go.Figure()
v = np.abs (coefficients)


def exp_cdf (x, a):
    return 1- np.exp(-a*x)

popt, pcov = opt.curve_fit(exp_cdf, np.sort(v), np.linspace(0, 1, len(v)), p0=[1])

thr = -np.log(1 - 1/len(coefficients))/popt[0]


fig.add_trace(go.Scatter (x = np.sort(v), y = np.linspace (0,1,len(v)), mode = 'markers'))

c = np.linspace (0, 18, 100)
fig.add_trace(go.Scatter (x = c, y = exp_cdf(c, *popt)))
fig.add_trace(go.Scatter (x = (thr+1,thr+1), y = (0.1, 0.99)))


In [14]:
popt

array([0.90384146])

In [111]:
fig = go.Figure()
fig.add_trace(go.Scatter (x = np.sort(np.log(v)), y = np.linspace (0,1,len(v)), mode = 'markers'))